In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import random_split, DataLoader
import time
from tqdm import tqdm
import seaborn as sns
from initializations import initialize_model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [24]:
# Define a CNN model with configurable activation functions for each layer
class CNNModel(nn.Module):
    def __init__(self, activations=None):
        super(CNNModel, self).__init__()

        if activations is None:
            activations = nn.ReLU()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1, stride=1),
            activations,
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1),
            activations,
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, padding=1),
            activations,
            nn.MaxPool2d(2)
        )

        self.dense1 = nn.Sequential(
            nn.Linear(128, 128),
            activations
        )
        self.dense2 = nn.Sequential(
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.dense1(out)
        out = self.dense2(out)
        return out

In [27]:
def train_val_model(model, num_epochs=20):
    train={
        'loss': [],
        'time': []
    }
    validation={
        'accuracy': []
    }
    for epoch in range(num_epochs):
        model.train()
        start_time = time.time()
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)
        epoch_loss = 0
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix({'Loss': loss.item()})

            _, predicted = torch.max(outputs.data, 1)

        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Time: {time.time() - start_time:.2f}s')
        
        train['loss'].append(avg_loss)
        train['time'].append(time.time() - start_time)
        model.eval()
        predictions_val = []
        labels_val = []
        with torch.no_grad():
            for images, labels in validation_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                _, predicted = torch.max(outputs.data, 1)
                predictions_val.extend(predicted.cpu().numpy())
                labels_val.extend(labels.cpu().numpy())

        validation['accuracy'].append(accuracy_score(labels_val, predictions_val))

    return train, validation

In [ ]:
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [29]:
activations = [nn.ReLU(), nn.LeakyReLU(), nn.Sigmoid(), nn.Tanh(), nn.SELU()]
initialisations = ['normal', 'xavier_normal', 'he_normal', 'lecun_normal', 'zeros']

In [ ]:
results = {}
for i in range(5):
    model = CNNModel(activations[i]).to(device)
    for j in range(5):
        initialize_model(model, initialisations[j])
        train, validation = train_val_model(model, num_epochs=1)
        results[initialisations[j] + '_' + activations[i].__class__.__name__] = {
            'train': train,
            'validation': validation
        }

In [ ]:
# no stability check here - will just look at the visualizations

convergence_speed = {}
finall_accuracy = {}

for key, value in results.items():
    max_accuracy = max(value['validation']['accuracy'])
    # iteration to surpass 90% of the highest accuracy
    index = next(i for i, v in enumerate(value['validation']['accuracy']) if v > 0.9 * max_accuracy)
    convergence_speed[key] = sum(value['train']['time'][:index])

    finall_accuracy[key] = max_accuracy

In [ ]:
"""Plots for Each Combination:
Training Loss vs. Epochs.
Validation Accuracy vs. Epochs.
"""

for key, value in results.items():
    fig, axs = plt.subplots(2)
    fig.suptitle(key)
    axs[0].plot(value['train']['loss'])
    axs[0].set_title('Training Loss')
    axs[1].plot(value['validation']['accuracy'])
    axs[1].set_title('Validation Accuracy')
    plt.savefig(f'plots/{key}.png')
    plt.show()

In [ ]:
"""Heatmaps:
Accuracy for each combination of initialization and activation function.
Training time for each combination.
"""

accuracy = []
time = []
for i in range(5):
    accuracy.append([finall_accuracy[initialisations[j] + '_' + activations[i].__class__.__name__] for j in range(5)])
    time.append([convergence_speed[initialisations[j] + '_' + activations[i].__class__.__name__] for j in range(5)])

fig, axs = plt.subplots(2)
fig.suptitle('Accuracy')

sns.heatmap(accuracy, annot=True, xticklabels=initialisations, yticklabels=[activation.__class__.__name__ for activation in activations], ax=axs[0])
sns.heatmap(time, annot=True, xticklabels=initialisations, yticklabels=[activation.__class__.__name__ for activation in activations], ax=axs[1])

plt.savefig('plots/heatmap.png')
plt.show()